## Preprocessing

In [56]:
import ujson as json
import glob
import os
import re

import numpy as np
import pandas as pd

from tqdm import tqdm as tqdm_notebook
import gc

%matplotlib inline
import time
import matplotlib.pyplot as plt

from joblib import Parallel,delayed
from itertools import islice

from collections import Counter
import nltk
from nltk.tokenize import word_tokenize

from itertools import zip_longest

sent_tokenizer = nltk.data.load('tokenizers/punkt/finnish.pickle')

import string

from nltk.tokenize import TweetTokenizer

import logging
logger = logging.getLogger(__name__)
logger.handlers = []
sh = logging.StreamHandler()
logger.setLevel(logging.DEBUG)
logger.addHandler(sh)

In [ ]:
def preprocess_lines(lines, tokenizer, min_sent_len=5):
    """Preprocess given lines."""
    contents = [c 
                for js in json.loads('[' + ','.join(lines) + ']')
                for c in js['content']]
    
    filter_re = re.compile(r'[^\w\s]')
    url_re = re.compile(r'\w+:\/\/\S*')
    
    sents = []
    for doc in contents:
        
        for sent in sent_tokenizer.tokenize(doc):
            sent_tokens = []
            
            # Additional RE
            sent = url_re.sub('<URL>', sent)
            
            # Tokenization
            tokens = tokenizer.tokenize(sent)

            # Cleaning up
            for token in tokens:
                normal_chars = filter_re.sub('', token)
                other_chars = filter_re.findall(token)
                        
                enough_normal_chars = len(normal_chars) > 0
                if enough_normal_chars:
                    sent_tokens.append(token)
                    continue
                    
                only_one_other = len(other_chars) == 1
                others_in_punct = any([c in string.punctuation for c in other_chars])
                if only_one_other and not others_in_punct and len(token) > 1:
                    sent_tokens.append(token)
            
            # Add to sentences
            if len(sent_tokens) >= min_sent_len:
                clean_sent = ' '.join(sent_tokens)
                sents.append(clean_sent)
    return sents

def grouper(iterable, n, fillvalue=None):
    """Returns iterator of certain length."""
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

def parallel_preprocess_lines(lines, tokenizer, n_jobs=3, min_sent_len=5):
    """Parallel preprocessing of lines.
    
    Args:
        lines (list): List of strings that are crawled lines and not in
            JSON format yet.
        tokenizer (object): 
        n_jobs (int): Number of parallel workers to use. Defaults to 3.
        min_sent_len (int): Minimum number of tokens to be considered
            as a sentence. Defaults to 5.
            
    Returns:
        List of unique sentences in an array.
    """
    n = int(np.ceil(len(lines) / n_jobs))
    job_lines = [lines[i:i + n] for i in range(0, len(lines), n)]
    sent_lists = Parallel(n_jobs=n_jobs)(
        delayed(preprocess_lines)(
            lines=lines,
            tokenizer=tokenizer,
            min_sent_len=min_sent_len
        )
        for lines in job_lines
    )
    sents = [sent for sent_list in sent_lists for sent in sent_list]
    return pd.unique(sents)
      
def process_file(filepath, tokenizer, out_filepath='../data/processed/test.sl',
                 mode='a', lines_per_chunk=30000, min_sent_len=5, n_jobs=3):
    with open(filepath, 'r', encoding='utf8') as f:
        logger.info('Reading number of lines in the file...')
        n_total_lines = sum(1 for _ in f)
        f.seek(0)
        
        with open(out_filepath, mode=mode, encoding='utf8') as fout:
            it = grouper(f, lines_per_chunk)
            for i,lines in enumerate(it):
                start_time = time.perf_counter()
                end_line = min((i + 1) * lines_per_chunk, n_total_lines) / 1e3
                logger.info(f'\nLines {i * lines_per_chunk / 1e3:.0f}'
                            f' - {end_line:.0f}k'
                            f'/ {n_total_lines / 1e3:.0f}k')
                
                sents = parallel_preprocess_lines(
                    [l for l in lines if l], n_jobs=n_jobs,
                    tokenizer=tokenizer, min_sent_len=min_sent_len)
                
                logger.info(f'Writing {len(sents)} sentences...')
                fout.write('\n'.join(sents))
                
                time_passed = time.perf_counter() - start_time
                logger.info(f'Chunk done in {time_passed:.0f} seconds!')
                gc.collect()
                
def process_all_files(in_filedir='../data/feed/',
                      out_filepath='../data/processed/all.sl',
                      lines_per_chunk=30000,
                      create_uncased=True,
                      min_sent_len=5,
                      tokenizer='tweet',
                      n_jobs=3):
    start_time = time.perf_counter()
    
    filepaths = [os.path.abspath(p) for p in glob.glob(in_filedir + '*.jl')]
    
    out_filepath = os.path.abspath(out_filepath)
    out_dir = os.path.dirname(out_filepath)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
        logger.warn(f'Created directory in {out_dir}')
    
    if tokenizer == 'tweet':
        tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True, 
                                   preserve_case=True)
    else:
        raise ValueError('Currently only "tweet" tokenizer is supported!')
    
    for i,path in enumerate(filepaths):
        logger.info(f'\n\nProcessing file "{path}" '
                    f'({i + 1} / {len(filepaths)})')
        process_file(path, mode='w' if i == 0 else 'a',
                     lines_per_chunk=lines_per_chunk,
                     out_filepath=out_filepath, min_sent_len=min_sent_len,
                     tokenizer=tokenizer,
                     n_jobs=n_jobs)
        
    if create_uncased:
        uncased_filepath = os.path.join(
            os.path.dirname(out_filepath),
            os.path.splitext(out_filepath)[0] + '_uncased.sl')
        logger.info(f'Creating uncased into "{uncased_filepath}"...')
        with open(out_filepath, 'r', encoding='utf8') as f:
            with open(uncased_filepath, 'w', encoding='utf8') as fout:
                for line in f:
                    fout.write(line.lower())
                    
    logger.info(f'All done in {time.perf_counter() - start_time:.0f} seconds!')
    
    
process_all_files()



Processing file "D:\Koodaus\EclipseWS\finnish-word-embeddings\data\feed\demi.jl" (1 / 11)
Reading number of lines in the file...

Lines 0 - 4k/ 4k
Writing 20550 sentences...
Chunk done in 3 seconds!


Processing file "D:\Koodaus\EclipseWS\finnish-word-embeddings\data\feed\iltalehti.jl" (2 / 11)
Reading number of lines in the file...

Lines 0 - 30k/ 74k


In [6]:
with open('./fwe/data/preprocessed/sents2.csv', 'r', encoding='utf8') as f:
    with open('./fwe/data/preprocessed/sents2_lowercase.csv', 'w', encoding='utf8') as fout:
        for line in f:
            fout.write(line.lower() + '\n')

## Word2Vec

In [13]:
from gensim.models import Word2Vec

In [14]:
from gensim.models.word2vec import LineSentence

In [15]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [19]:
sents = LineSentence('./fwe/data/preprocessed/sents2_lowercase.csv')

In [20]:
w2v = Word2Vec(
    window=5,
    size=100,
    min_count=5,
    max_vocab_size=None,
    workers=4
)

In [21]:
w2v.build_vocab(sents, progress_per=1e6)

2019-06-29 14:55:25,888 : INFO : collecting all words and their counts
2019-06-29 14:55:25,889 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-29 14:55:31,623 : INFO : PROGRESS: at sentence #1000000, processed 11362602 words, keeping 860000 word types
2019-06-29 14:55:37,804 : INFO : PROGRESS: at sentence #2000000, processed 23614632 words, keeping 1365131 word types
2019-06-29 14:55:43,615 : INFO : PROGRESS: at sentence #3000000, processed 34863876 words, keeping 1784125 word types
2019-06-29 14:55:49,465 : INFO : PROGRESS: at sentence #4000000, processed 46054446 words, keeping 2225887 word types
2019-06-29 14:55:55,544 : INFO : PROGRESS: at sentence #5000000, processed 57188657 words, keeping 2539791 word types
2019-06-29 14:56:01,459 : INFO : PROGRESS: at sentence #6000000, processed 67647947 words, keeping 2817251 word types
2019-06-29 14:56:07,242 : INFO : PROGRESS: at sentence #7000000, processed 78025070 words, keeping 3062750 word types
2019-

2019-06-29 15:02:06,019 : INFO : PROGRESS: at sentence #68000000, processed 758990134 words, keeping 15886174 word types
2019-06-29 15:02:11,743 : INFO : PROGRESS: at sentence #69000000, processed 770486718 words, keeping 15961845 word types
2019-06-29 15:02:17,428 : INFO : PROGRESS: at sentence #70000000, processed 782049391 words, keeping 16026168 word types
2019-06-29 15:02:23,190 : INFO : PROGRESS: at sentence #71000000, processed 793639884 words, keeping 16153396 word types
2019-06-29 15:02:28,951 : INFO : PROGRESS: at sentence #72000000, processed 805226256 words, keeping 16282003 word types
2019-06-29 15:02:34,715 : INFO : PROGRESS: at sentence #73000000, processed 816791482 words, keeping 16381730 word types
2019-06-29 15:02:40,488 : INFO : PROGRESS: at sentence #74000000, processed 828418512 words, keeping 16474918 word types
2019-06-29 15:02:46,271 : INFO : PROGRESS: at sentence #75000000, processed 840007359 words, keeping 16595729 word types
2019-06-29 15:02:52,058 : INFO :

In [22]:
w2v.train(
    sents,
    total_examples=w2v.corpus_count,
    epochs=w2v.epochs,
    queue_factor=2
)

2019-06-29 15:05:12,361 : INFO : training model with 4 workers on 3046105 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2019-06-29 15:05:13,371 : INFO : EPOCH 1 - PROGRESS: at 0.04% examples, 308477 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:05:14,372 : INFO : EPOCH 1 - PROGRESS: at 0.14% examples, 566304 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:05:15,381 : INFO : EPOCH 1 - PROGRESS: at 0.25% examples, 662336 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:05:16,594 : INFO : EPOCH 1 - PROGRESS: at 0.34% examples, 653777 words/s, in_qsize 0, out_qsize 1
2019-06-29 15:05:17,594 : INFO : EPOCH 1 - PROGRESS: at 0.43% examples, 693775 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:05:18,602 : INFO : EPOCH 1 - PROGRESS: at 0.52% examples, 724711 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:05:19,611 : INFO : EPOCH 1 - PROGRESS: at 0.61% examples, 742035 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:05:20,624 : INFO : EPOCH 1 - PROGRESS: at 0.69% ex

2019-06-29 15:06:26,140 : INFO : EPOCH 1 - PROGRESS: at 6.51% examples, 823097 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:06:27,150 : INFO : EPOCH 1 - PROGRESS: at 6.61% examples, 823547 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:06:28,156 : INFO : EPOCH 1 - PROGRESS: at 6.71% examples, 823914 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:06:29,167 : INFO : EPOCH 1 - PROGRESS: at 6.78% examples, 820992 words/s, in_qsize 2, out_qsize 0
2019-06-29 15:06:30,184 : INFO : EPOCH 1 - PROGRESS: at 6.88% examples, 821324 words/s, in_qsize 0, out_qsize 1
2019-06-29 15:06:31,184 : INFO : EPOCH 1 - PROGRESS: at 6.97% examples, 821042 words/s, in_qsize 3, out_qsize 0
2019-06-29 15:06:32,192 : INFO : EPOCH 1 - PROGRESS: at 7.07% examples, 821437 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:06:33,192 : INFO : EPOCH 1 - PROGRESS: at 7.17% examples, 821866 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:06:34,207 : INFO : EPOCH 1 - PROGRESS: at 7.26% examples, 821942 words/s, in_qsize 0, out_

2019-06-29 15:07:39,666 : INFO : EPOCH 1 - PROGRESS: at 13.38% examples, 822748 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:07:40,667 : INFO : EPOCH 1 - PROGRESS: at 13.47% examples, 823009 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:07:41,679 : INFO : EPOCH 1 - PROGRESS: at 13.57% examples, 822936 words/s, in_qsize 1, out_qsize 0
2019-06-29 15:07:42,689 : INFO : EPOCH 1 - PROGRESS: at 13.66% examples, 823236 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:07:43,704 : INFO : EPOCH 1 - PROGRESS: at 13.75% examples, 823145 words/s, in_qsize 0, out_qsize 1
2019-06-29 15:07:44,712 : INFO : EPOCH 1 - PROGRESS: at 13.85% examples, 823483 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:07:45,724 : INFO : EPOCH 1 - PROGRESS: at 13.94% examples, 824081 words/s, in_qsize 1, out_qsize 0
2019-06-29 15:07:46,727 : INFO : EPOCH 1 - PROGRESS: at 14.03% examples, 824240 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:07:47,743 : INFO : EPOCH 1 - PROGRESS: at 14.12% examples, 825251 words/s, in_qsiz

2019-06-29 15:08:53,182 : INFO : EPOCH 1 - PROGRESS: at 20.14% examples, 828892 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:08:54,184 : INFO : EPOCH 1 - PROGRESS: at 20.23% examples, 828876 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:08:55,201 : INFO : EPOCH 1 - PROGRESS: at 20.32% examples, 828941 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:08:56,209 : INFO : EPOCH 1 - PROGRESS: at 20.41% examples, 828954 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:08:57,226 : INFO : EPOCH 1 - PROGRESS: at 20.48% examples, 827743 words/s, in_qsize 6, out_qsize 0
2019-06-29 15:08:58,233 : INFO : EPOCH 1 - PROGRESS: at 20.58% examples, 827991 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:08:59,241 : INFO : EPOCH 1 - PROGRESS: at 20.67% examples, 827840 words/s, in_qsize 5, out_qsize 2
2019-06-29 15:09:00,249 : INFO : EPOCH 1 - PROGRESS: at 20.77% examples, 828325 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:09:01,255 : INFO : EPOCH 1 - PROGRESS: at 20.87% examples, 828418 words/s, in_qsiz

2019-06-29 15:10:06,944 : INFO : EPOCH 1 - PROGRESS: at 27.35% examples, 827802 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:10:07,950 : INFO : EPOCH 1 - PROGRESS: at 27.45% examples, 827875 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:10:08,960 : INFO : EPOCH 1 - PROGRESS: at 27.55% examples, 827803 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:10:09,965 : INFO : EPOCH 1 - PROGRESS: at 27.65% examples, 827889 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:10:10,967 : INFO : EPOCH 1 - PROGRESS: at 27.76% examples, 828059 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:10:11,973 : INFO : EPOCH 1 - PROGRESS: at 27.87% examples, 827454 words/s, in_qsize 2, out_qsize 0
2019-06-29 15:10:12,974 : INFO : EPOCH 1 - PROGRESS: at 27.99% examples, 827686 words/s, in_qsize 0, out_qsize 0
2019-06-29 15:10:13,991 : INFO : EPOCH 1 - PROGRESS: at 28.10% examples, 827779 words/s, in_qsize 4, out_qsize 0
2019-06-29 15:10:15,015 : INFO : EPOCH 1 - PROGRESS: at 28.22% examples, 827945 words/s, in_qsiz

...

2019-06-29 16:33:23,497 : INFO : EPOCH 5 - PROGRESS: at 78.46% examples, 853658 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:33:24,503 : INFO : EPOCH 5 - PROGRESS: at 78.55% examples, 853724 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:33:25,510 : INFO : EPOCH 5 - PROGRESS: at 78.65% examples, 853770 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:33:26,511 : INFO : EPOCH 5 - PROGRESS: at 78.74% examples, 853802 words/s, in_qsize 3, out_qsize 0
2019-06-29 16:33:27,516 : INFO : EPOCH 5 - PROGRESS: at 78.83% examples, 853859 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:33:28,522 : INFO : EPOCH 5 - PROGRESS: at 78.93% examples, 853909 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:33:29,526 : INFO : EPOCH 5 - PROGRESS: at 79.02% examples, 853974 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:33:30,535 : INFO : EPOCH 5 - PROGRESS: at 79.12% examples, 854016 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:33:31,537 : INFO : EPOCH 5 - PROGRESS: at 79.21% examples, 854054 words/s, in_qsiz

2019-06-29 16:34:37,112 : INFO : EPOCH 5 - PROGRESS: at 85.35% examples, 856500 words/s, in_qsize 0, out_qsize 1
2019-06-29 16:34:38,117 : INFO : EPOCH 5 - PROGRESS: at 85.45% examples, 856569 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:34:39,122 : INFO : EPOCH 5 - PROGRESS: at 85.55% examples, 856627 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:34:40,129 : INFO : EPOCH 5 - PROGRESS: at 85.64% examples, 856663 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:34:41,141 : INFO : EPOCH 5 - PROGRESS: at 85.74% examples, 856718 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:34:42,153 : INFO : EPOCH 5 - PROGRESS: at 85.83% examples, 856750 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:34:43,158 : INFO : EPOCH 5 - PROGRESS: at 85.93% examples, 856798 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:34:44,159 : INFO : EPOCH 5 - PROGRESS: at 86.03% examples, 856855 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:34:45,162 : INFO : EPOCH 5 - PROGRESS: at 86.12% examples, 856901 words/s, in_qsiz

2019-06-29 16:35:50,601 : INFO : EPOCH 5 - PROGRESS: at 92.26% examples, 858791 words/s, in_qsize 2, out_qsize 0
2019-06-29 16:35:51,606 : INFO : EPOCH 5 - PROGRESS: at 92.35% examples, 858819 words/s, in_qsize 0, out_qsize 1
2019-06-29 16:35:52,628 : INFO : EPOCH 5 - PROGRESS: at 92.45% examples, 858853 words/s, in_qsize 0, out_qsize 1
2019-06-29 16:35:53,628 : INFO : EPOCH 5 - PROGRESS: at 92.55% examples, 858889 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:35:54,633 : INFO : EPOCH 5 - PROGRESS: at 92.64% examples, 858933 words/s, in_qsize 1, out_qsize 0
2019-06-29 16:35:55,641 : INFO : EPOCH 5 - PROGRESS: at 92.74% examples, 858972 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:35:56,647 : INFO : EPOCH 5 - PROGRESS: at 92.84% examples, 859010 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:35:57,650 : INFO : EPOCH 5 - PROGRESS: at 92.94% examples, 859048 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:35:58,660 : INFO : EPOCH 5 - PROGRESS: at 93.00% examples, 858837 words/s, in_qsiz

2019-06-29 16:37:04,257 : INFO : EPOCH 5 - PROGRESS: at 99.31% examples, 860456 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:37:05,523 : INFO : EPOCH 5 - PROGRESS: at 99.42% examples, 860224 words/s, in_qsize 0, out_qsize 1
2019-06-29 16:37:06,534 : INFO : EPOCH 5 - PROGRESS: at 99.54% examples, 860224 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:37:07,547 : INFO : EPOCH 5 - PROGRESS: at 99.66% examples, 860232 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:37:08,553 : INFO : EPOCH 5 - PROGRESS: at 99.78% examples, 860244 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:37:09,556 : INFO : EPOCH 5 - PROGRESS: at 99.89% examples, 860235 words/s, in_qsize 0, out_qsize 0
2019-06-29 16:37:10,523 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-06-29 16:37:10,526 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-06-29 16:37:10,537 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-06-29 16:37:10,541 : INFO : worker thread fi

(4621385254, 5140108470)

In [57]:
w2v.wv.most_similar('vittu')

[('jumalauta', 0.9041902422904968),
 ('v*ttu', 0.8968625068664551),
 ('vttu', 0.8679633736610413),
 ('v***u', 0.8617925047874451),
 ('v****', 0.8465098738670349),
 ('viddu', 0.8457162380218506),
 ('perkele', 0.8328149318695068),
 ('vi**u', 0.828162670135498),
 ('saatana', 0.8281378746032715),
 ('vitttu', 0.8233478665351868)]

In [24]:
w2v.save('./fwe/data/preprocessed/w2v2_lowercase.pkl')

2019-06-29 16:37:12,903 : INFO : saving Word2Vec object under ./fwe/data/preprocessed/w2v2_lowercase.pkl, separately None
2019-06-29 16:37:12,904 : INFO : storing np array 'vectors' to ./fwe/data/preprocessed/w2v2_lowercase.pkl.wv.vectors.npy
2019-06-29 16:37:23,254 : INFO : not storing attribute vectors_norm
2019-06-29 16:37:23,258 : INFO : storing np array 'syn1neg' to ./fwe/data/preprocessed/w2v2_lowercase.pkl.trainables.syn1neg.npy
2019-06-29 16:37:33,441 : INFO : not storing attribute cum_table
D:\Koodaus\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-06-29 16:37:40,552 : INFO : saved ./fwe/data/preprocessed/w2v2_lowercase.pkl


In [26]:
w2v.wv.save_word2vec_format('./fwe/data/preprocessed/w2v2_lowercase.vec', binary=False)

2019-06-29 16:42:30,692 : INFO : storing 3046105x100 projection weights into ./fwe/data/preprocessed/w2v2_lowercase.vec
D:\Koodaus\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## FastText

In [31]:
from gensim.models import FastText

In [32]:
ft = FastText(
    size=100,
    window=5,
    min_count=5,
    workers=4
)

2019-06-29 16:55:46,953 : INFO : resetting layer weights
2019-06-29 16:55:46,953 : INFO : Total number of ngrams is 0


In [33]:
ft.build_vocab(sents, progress_per=1e6)

2019-06-29 16:55:49,631 : INFO : collecting all words and their counts
2019-06-29 16:55:49,632 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-29 16:55:55,323 : INFO : PROGRESS: at sentence #1000000, processed 11362602 words, keeping 860000 word types
2019-06-29 16:56:01,421 : INFO : PROGRESS: at sentence #2000000, processed 23614632 words, keeping 1365131 word types
2019-06-29 16:56:07,170 : INFO : PROGRESS: at sentence #3000000, processed 34863876 words, keeping 1784125 word types
2019-06-29 16:56:12,977 : INFO : PROGRESS: at sentence #4000000, processed 46054446 words, keeping 2225887 word types
2019-06-29 16:56:18,961 : INFO : PROGRESS: at sentence #5000000, processed 57188657 words, keeping 2539791 word types
2019-06-29 16:56:24,657 : INFO : PROGRESS: at sentence #6000000, processed 67647947 words, keeping 2817251 word types
2019-06-29 16:56:30,125 : INFO : PROGRESS: at sentence #7000000, processed 78025070 words, keeping 3062750 word types
2019-

2019-06-29 17:02:24,562 : INFO : PROGRESS: at sentence #68000000, processed 758990134 words, keeping 15886174 word types
2019-06-29 17:02:30,642 : INFO : PROGRESS: at sentence #69000000, processed 770486718 words, keeping 15961845 word types
2019-06-29 17:02:37,142 : INFO : PROGRESS: at sentence #70000000, processed 782049391 words, keeping 16026168 word types
2019-06-29 17:02:43,699 : INFO : PROGRESS: at sentence #71000000, processed 793639884 words, keeping 16153396 word types
2019-06-29 17:02:50,189 : INFO : PROGRESS: at sentence #72000000, processed 805226256 words, keeping 16282003 word types
2019-06-29 17:02:56,464 : INFO : PROGRESS: at sentence #73000000, processed 816791482 words, keeping 16381730 word types
2019-06-29 17:03:02,331 : INFO : PROGRESS: at sentence #74000000, processed 828418512 words, keeping 16474918 word types
2019-06-29 17:03:08,167 : INFO : PROGRESS: at sentence #75000000, processed 840007359 words, keeping 16595729 word types
2019-06-29 17:03:14,128 : INFO :

In [34]:
ft.train(
    sents,
    total_examples=ft.corpus_count,
    epochs=ft.epochs,
    queue_factor=2
)

2019-06-29 18:01:49,811 : INFO : training model with 4 workers on 3046105 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2019-06-29 18:01:50,871 : INFO : EPOCH 1 - PROGRESS: at 0.04% examples, 294150 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:01:51,923 : INFO : EPOCH 1 - PROGRESS: at 0.08% examples, 306447 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:01:52,939 : INFO : EPOCH 1 - PROGRESS: at 0.13% examples, 320131 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:01:54,575 : INFO : EPOCH 1 - PROGRESS: at 0.17% examples, 284745 words/s, in_qsize 11, out_qsize 3
2019-06-29 18:01:55,575 : INFO : EPOCH 1 - PROGRESS: at 0.21% examples, 295764 words/s, in_qsize 14, out_qsize 0
2019-06-29 18:01:56,599 : INFO : EPOCH 1 - PROGRESS: at 0.25% examples, 301166 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:01:57,625 : INFO : EPOCH 1 - PROGRESS: at 0.30% examples, 308401 words/s, in_qsize 16, out_qsize 0
2019-06-29 18:01:58,642 : INFO : EPOCH 1 - PROGRESS: at 0

2019-06-29 18:03:05,519 : INFO : EPOCH 1 - PROGRESS: at 2.58% examples, 331343 words/s, in_qsize 14, out_qsize 0
2019-06-29 18:03:06,522 : INFO : EPOCH 1 - PROGRESS: at 2.61% examples, 331301 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:03:07,561 : INFO : EPOCH 1 - PROGRESS: at 2.65% examples, 331514 words/s, in_qsize 15, out_qsize 1
2019-06-29 18:03:08,563 : INFO : EPOCH 1 - PROGRESS: at 2.69% examples, 331793 words/s, in_qsize 16, out_qsize 0
2019-06-29 18:03:09,566 : INFO : EPOCH 1 - PROGRESS: at 2.73% examples, 331966 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:03:10,576 : INFO : EPOCH 1 - PROGRESS: at 2.76% examples, 332093 words/s, in_qsize 15, out_qsize 1
2019-06-29 18:03:11,595 : INFO : EPOCH 1 - PROGRESS: at 2.80% examples, 332196 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:03:12,628 : INFO : EPOCH 1 - PROGRESS: at 2.83% examples, 332221 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:03:13,679 : INFO : EPOCH 1 - PROGRESS: at 2.87% examples, 332228 words/s, in_qsize

2019-06-29 18:04:20,187 : INFO : EPOCH 1 - PROGRESS: at 5.02% examples, 316325 words/s, in_qsize 16, out_qsize 0
2019-06-29 18:04:21,205 : INFO : EPOCH 1 - PROGRESS: at 5.05% examples, 315993 words/s, in_qsize 16, out_qsize 0
2019-06-29 18:04:22,206 : INFO : EPOCH 1 - PROGRESS: at 5.08% examples, 315772 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:04:23,206 : INFO : EPOCH 1 - PROGRESS: at 5.11% examples, 315552 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:04:24,206 : INFO : EPOCH 1 - PROGRESS: at 5.15% examples, 315374 words/s, in_qsize 16, out_qsize 0
2019-06-29 18:04:25,232 : INFO : EPOCH 1 - PROGRESS: at 5.18% examples, 315186 words/s, in_qsize 14, out_qsize 1
2019-06-29 18:04:26,270 : INFO : EPOCH 1 - PROGRESS: at 5.21% examples, 314908 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:04:27,301 : INFO : EPOCH 1 - PROGRESS: at 5.25% examples, 314937 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:04:28,325 : INFO : EPOCH 1 - PROGRESS: at 5.29% examples, 315032 words/s, in_qsize

2019-06-29 18:05:35,205 : INFO : EPOCH 1 - PROGRESS: at 7.79% examples, 318565 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:05:36,217 : INFO : EPOCH 1 - PROGRESS: at 7.83% examples, 318710 words/s, in_qsize 14, out_qsize 0
2019-06-29 18:05:37,230 : INFO : EPOCH 1 - PROGRESS: at 7.87% examples, 318733 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:05:38,243 : INFO : EPOCH 1 - PROGRESS: at 7.91% examples, 318796 words/s, in_qsize 16, out_qsize 0
2019-06-29 18:05:39,282 : INFO : EPOCH 1 - PROGRESS: at 7.95% examples, 318936 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:05:40,312 : INFO : EPOCH 1 - PROGRESS: at 7.98% examples, 318938 words/s, in_qsize 14, out_qsize 1
2019-06-29 18:05:41,314 : INFO : EPOCH 1 - PROGRESS: at 8.02% examples, 318979 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:05:42,327 : INFO : EPOCH 1 - PROGRESS: at 8.06% examples, 319004 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:05:43,334 : INFO : EPOCH 1 - PROGRESS: at 8.10% examples, 319038 words/s, in_qsize

2019-06-29 18:06:49,988 : INFO : EPOCH 1 - PROGRESS: at 10.50% examples, 319701 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:06:51,437 : INFO : EPOCH 1 - PROGRESS: at 10.53% examples, 319131 words/s, in_qsize 11, out_qsize 3
2019-06-29 18:06:52,454 : INFO : EPOCH 1 - PROGRESS: at 10.57% examples, 319202 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:06:53,461 : INFO : EPOCH 1 - PROGRESS: at 10.61% examples, 319255 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:06:54,490 : INFO : EPOCH 1 - PROGRESS: at 10.66% examples, 319419 words/s, in_qsize 15, out_qsize 1
2019-06-29 18:06:55,525 : INFO : EPOCH 1 - PROGRESS: at 10.70% examples, 319454 words/s, in_qsize 16, out_qsize 0
2019-06-29 18:06:56,526 : INFO : EPOCH 1 - PROGRESS: at 10.74% examples, 319464 words/s, in_qsize 15, out_qsize 0
2019-06-29 18:06:57,555 : INFO : EPOCH 1 - PROGRESS: at 10.77% examples, 319445 words/s, in_qsize 16, out_qsize 0
2019-06-29 18:06:58,567 : INFO : EPOCH 1 - PROGRESS: at 10.81% examples, 319443 words/s,

...

2019-06-29 21:38:02,383 : INFO : EPOCH 5 - PROGRESS: at 86.81% examples, 331596 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:38:03,406 : INFO : EPOCH 5 - PROGRESS: at 86.84% examples, 331590 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:38:04,408 : INFO : EPOCH 5 - PROGRESS: at 86.88% examples, 331592 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:38:05,429 : INFO : EPOCH 5 - PROGRESS: at 86.92% examples, 331596 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:38:06,435 : INFO : EPOCH 5 - PROGRESS: at 86.96% examples, 331603 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:38:07,435 : INFO : EPOCH 5 - PROGRESS: at 86.99% examples, 331603 words/s, in_qsize 14, out_qsize 0
2019-06-29 21:38:08,472 : INFO : EPOCH 5 - PROGRESS: at 87.03% examples, 331594 words/s, in_qsize 16, out_qsize 0
2019-06-29 21:38:09,489 : INFO : EPOCH 5 - PROGRESS: at 87.06% examples, 331591 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:38:10,508 : INFO : EPOCH 5 - PROGRESS: at 87.10% examples, 331587 words/s,

2019-06-29 21:39:16,685 : INFO : EPOCH 5 - PROGRESS: at 89.37% examples, 331421 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:39:17,701 : INFO : EPOCH 5 - PROGRESS: at 89.41% examples, 331425 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:39:18,741 : INFO : EPOCH 5 - PROGRESS: at 89.45% examples, 331422 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:39:19,795 : INFO : EPOCH 5 - PROGRESS: at 89.48% examples, 331421 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:39:20,795 : INFO : EPOCH 5 - PROGRESS: at 89.52% examples, 331420 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:39:21,796 : INFO : EPOCH 5 - PROGRESS: at 89.55% examples, 331412 words/s, in_qsize 16, out_qsize 0
2019-06-29 21:39:22,806 : INFO : EPOCH 5 - PROGRESS: at 89.58% examples, 331406 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:39:23,812 : INFO : EPOCH 5 - PROGRESS: at 89.62% examples, 331401 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:39:24,859 : INFO : EPOCH 5 - PROGRESS: at 89.65% examples, 331393 words/s,

2019-06-29 21:40:30,522 : INFO : EPOCH 5 - PROGRESS: at 91.89% examples, 331186 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:40:31,529 : INFO : EPOCH 5 - PROGRESS: at 91.93% examples, 331186 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:40:32,539 : INFO : EPOCH 5 - PROGRESS: at 91.97% examples, 331185 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:40:33,550 : INFO : EPOCH 5 - PROGRESS: at 92.01% examples, 331188 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:40:34,575 : INFO : EPOCH 5 - PROGRESS: at 92.04% examples, 331184 words/s, in_qsize 16, out_qsize 0
2019-06-29 21:40:35,586 : INFO : EPOCH 5 - PROGRESS: at 92.08% examples, 331189 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:40:36,609 : INFO : EPOCH 5 - PROGRESS: at 92.11% examples, 331189 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:40:37,618 : INFO : EPOCH 5 - PROGRESS: at 92.15% examples, 331186 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:40:38,650 : INFO : EPOCH 5 - PROGRESS: at 92.18% examples, 331183 words/s,

2019-06-29 21:41:44,249 : INFO : EPOCH 5 - PROGRESS: at 94.42% examples, 331095 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:41:45,259 : INFO : EPOCH 5 - PROGRESS: at 94.45% examples, 331084 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:41:46,285 : INFO : EPOCH 5 - PROGRESS: at 94.48% examples, 331081 words/s, in_qsize 16, out_qsize 0
2019-06-29 21:41:47,289 : INFO : EPOCH 5 - PROGRESS: at 94.52% examples, 331089 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:41:48,313 : INFO : EPOCH 5 - PROGRESS: at 94.56% examples, 331100 words/s, in_qsize 16, out_qsize 0
2019-06-29 21:41:49,321 : INFO : EPOCH 5 - PROGRESS: at 94.60% examples, 331102 words/s, in_qsize 15, out_qsize 1
2019-06-29 21:41:50,332 : INFO : EPOCH 5 - PROGRESS: at 94.63% examples, 331098 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:41:51,391 : INFO : EPOCH 5 - PROGRESS: at 94.67% examples, 331101 words/s, in_qsize 16, out_qsize 0
2019-06-29 21:41:52,411 : INFO : EPOCH 5 - PROGRESS: at 94.71% examples, 331103 words/s,

2019-06-29 21:42:58,250 : INFO : EPOCH 5 - PROGRESS: at 97.16% examples, 331604 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:42:59,297 : INFO : EPOCH 5 - PROGRESS: at 97.21% examples, 331623 words/s, in_qsize 16, out_qsize 0
2019-06-29 21:43:00,329 : INFO : EPOCH 5 - PROGRESS: at 97.25% examples, 331637 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:43:01,363 : INFO : EPOCH 5 - PROGRESS: at 97.29% examples, 331658 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:43:02,368 : INFO : EPOCH 5 - PROGRESS: at 97.33% examples, 331676 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:43:03,376 : INFO : EPOCH 5 - PROGRESS: at 97.37% examples, 331691 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:43:04,382 : INFO : EPOCH 5 - PROGRESS: at 97.41% examples, 331705 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:43:05,387 : INFO : EPOCH 5 - PROGRESS: at 97.45% examples, 331719 words/s, in_qsize 15, out_qsize 0
2019-06-29 21:43:06,405 : INFO : EPOCH 5 - PROGRESS: at 97.49% examples, 331731 words/s,

In [36]:
ft.save('./fwe/data/preprocessed/ft2_lowercase.pkl')

2019-06-29 22:14:30,771 : INFO : saving FastText object under ./fwe/data/preprocessed/ft2_lowercase.pkl, separately None
2019-06-29 22:14:30,772 : INFO : storing np array 'vectors' to ./fwe/data/preprocessed/ft2_lowercase.pkl.wv.vectors.npy
2019-06-29 22:14:41,540 : INFO : not storing attribute vectors_norm
2019-06-29 22:14:41,544 : INFO : storing np array 'vectors_vocab' to ./fwe/data/preprocessed/ft2_lowercase.pkl.wv.vectors_vocab.npy
2019-06-29 22:14:51,799 : INFO : storing np array 'vectors_ngrams' to ./fwe/data/preprocessed/ft2_lowercase.pkl.wv.vectors_ngrams.npy
2019-06-29 22:14:57,659 : INFO : not storing attribute vectors_ngrams_norm
2019-06-29 22:14:57,662 : INFO : not storing attribute vectors_vocab_norm
2019-06-29 22:14:57,666 : INFO : not storing attribute buckets_word
2019-06-29 22:14:57,668 : INFO : storing np array 'syn1neg' to ./fwe/data/preprocessed/ft2_lowercase.pkl.trainables.syn1neg.npy
2019-06-29 22:15:07,928 : INFO : storing np array 'vectors_vocab_lockf' to ./fwe

In [56]:
ft.wv.save_word2vec_format('./fwe/data/preprocessed/ft2_lowercase.vec', binary=False)

2019-06-29 23:21:36,969 : INFO : storing 3046105x100 projection weights into ./fwe/data/preprocessed/ft2_lowercase.vec
D:\Koodaus\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## Random testing

In [37]:
from IPython.core.display import display

In [89]:
word = 'leijona'
topn = 40

display(ft.wv.most_similar(word, topn=topn))
w2v.wv.most_similar(word, topn=topn)

[('.leijona', 0.9694546461105347),
 ("'leijona", 0.9675278663635254),
 ('*leijona', 0.9650520086288452),
 ('rapuleijona', 0.958706259727478),
 ('leijonaemona', 0.9577534198760986),
 ('pikkuleijona', 0.9410644769668579),
 ('3.leijona', 0.9402765035629272),
 ('urosleijona', 0.9388136863708496),
 ('naisleijona', 0.93558669090271),
 ('venus-leijona', 0.9350501298904419),
 ('kuu-leijona', 0.9338263273239136),
 ('leijona-oinas', 0.9317204356193542),
 ('vaaka-leijona', 0.9284749031066895),
 ('leijonana', 0.9279550909996033),
 ('emoleijona', 0.9267336130142212),
 ('leijona/härkä', 0.9265311360359192),
 ('mars-leijona', 0.921281099319458),
 ('naarasleijona', 0.9208881855010986),
 ('leijonapoika', 0.9203804731369019),
 ('leijonakippari', 0.9196872711181641),
 ('leijona-sika', 0.9194720387458801),
 ('kulta-leijona', 0.919461190700531),
 ('suomileijona', 0.9171470403671265),
 ('sankarileijona', 0.915271520614624),
 ('sirkusleijona', 0.9152035117149353),
 ('leijonanaaras', 0.9149506092071533),
 ('o

[('härkä', 0.877738356590271),
 ('kauris', 0.8104618191719055),
 ('rapu', 0.8062412738800049),
 ('puhveli', 0.7826233506202698),
 ('skorpioni', 0.7814520597457886),
 ('jousimies', 0.7677141427993774),
 ('vesimies', 0.754247784614563),
 ('tiikeri', 0.7503178715705872),
 ('oinas', 0.746002197265625),
 ('skorppari', 0.7142601013183594),
 ('lohikäärme', 0.7116947174072266),
 ('kaksonen', 0.7047980427742004),
 ('terrieri', 0.6899539232254028),
 ('skorppioni', 0.6893386840820312),
 ('dobberi', 0.6841057538986206),
 ('kaukkari', 0.6838588714599609),
 ('jouskari', 0.6784685850143433),
 ('kukko', 0.6763876080513),
 ('pantteri', 0.665902853012085),
 ('lammas', 0.6651714444160461),
 ('kirahvi', 0.6642903685569763),
 ('kuurapu', 0.6623742580413818),
 ('villakoira', 0.6612218618392944),
 ('kääpiö', 0.6607388257980347),
 ('kuukalat', 0.6604245901107788),
 ('bokseri', 0.6593948602676392),
 ('neitsyt', 0.6579749584197998),
 ('mopsi', 0.6540440320968628),
 ('alpakka', 0.6522607803344727),
 ('bernhardin

In [49]:
from gensim.models.keyedvectors import KeyedVectors
kv = KeyedVectors.load_word2vec_format('./fwe/data/preprocessed/w2v2_lowercase.vec')

2019-06-29 23:14:39,544 : INFO : loading projection weights from ./fwe/data/preprocessed/w2v2_lowercase.vec
D:\Koodaus\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-06-29 23:18:37,381 : INFO : loaded (3046105, 100) matrix from ./fwe/data/preprocessed/w2v2_lowercase.vec


In [60]:
len(kv.vocab)

3046105